# Implementação do Algoritmo Simplex 

## Descrição Geral
Este notebook apresenta uma implementação do algoritmo simplex.

---

### **Informações dos Autores**
- **Autores - Numero Usp**  

    Cody Stefano Barham Setti – 4856322

    Ian de Holanda Cavalcanti Bezerra – 13835412
                
    Julia Graziosi Ortiz – 11797810
                
    Katlyn Ribeiro Almeida – 14586070
                
    Matheus Araujo Pinheiro – 14676810
---

### **Objetivos**
O objetivo deste trabalho foi implementar o algoritmo simplex na linguagem de programação Python e
resolver com tal implementação uma gama de problemas de otimização linear.

---

### **Ferramentas e Bibliotecas Utilizadas**
Linguagem e bibliotecas específicas: como Python; NumPy, Math, Time:

---

In [55]:
# Bibliotecas
import math
import numpy as np
import time

---

### **Organização**
1. **Obter solução básica viável** 
2. **Aplicar Método Simplex:** 
3. **Simplex de duas fases:** 
4. **Testando alguns exemplos:**

---

## 1- Fase 1: Obter solução básica viável
Funções auxiliares para encontrar variáveis básicas:

In [56]:
# Verifica se uma coluna é uma coluna da matriz identidade.
# Uma coluna é da identidade se a soma dos elementos é 1 e 
# contém exatamente um '1' e o restante '0's.
def eh_basica(coluna):
    return sum(coluna) == 1 and len([c for c in coluna if c == 0]) == len(coluna) - 1

In [57]:
# Retorna o índice da posição onde existe o valor '1' do vetor.
# Utilizado para identificar as variáveis básicas no tableau.
def get_one(vector):
    for i in range(len(vector)):
        if vector[i] == 1:
            return i

In [58]:
# Verifica se um elemento é considerado zero, dentro de uma tolerância (10e-4).
# Isso é necessário para evitar problemas de precisão em cálculos numéricos.
def is_zero(element):
    return abs(element) <= 10e-4

In [59]:
# Identifica as variáveis básicas e associa o índice das colunas que correspondem 
# a colunas da matriz identidade.
# Retorna:
# - `id`: um vetor identidade indicando se uma linha contém uma variável básica.
# - `i_B`: vetor contendo os índices das colunas que são variáveis básicas, ou -1 se não forem.
def basicas(A):
    id = [0] * len(A)       # Marca quais linhas já possuem variáveis básicas.
    i_B = [-1] * len(A)     # Índices das colunas básicas.
    colunas = np.array(A).T # Transpõe a matriz A para acessar as colunas.
    for coluna in range(0, len(colunas)):
        if eh_basica(colunas[coluna]) and id[get_one(colunas[coluna])] == 0:
            i_B[get_one(colunas[coluna])]= coluna
            id[get_one(colunas[coluna])] = 1
    return id, i_B
# OBS: quando escreve basicas(A)[0] --> acessa quais colunas
#                     basicas(A)[1] --> acessa os indices (-1) das basicas

Verifica se é necessário adicionar variáveis artificiais:

- se sim, então retorna os coeficientes do novo problema (minimizar o custo das artificiais)
- se não, então o problema segue o mesmo

In [60]:
# Implementa a Fase 1 do método simplex. Adiciona variáveis artificiais para garantir uma solução viável.
# Se as variáveis artificiais forem necessárias, elas são adicionadas às restrições e seus custos são definidos como 1.
# Retorna:
# - Custos atualizados (incluindo artificiais, se necessário).
# - Matriz A e vetor b modificados.
# - Um flag `d` indicando se variáveis artificiais foram adicionadas.
def fase1(c, A, b):
    nvar = len(A[0])
    d = (False, nvar)
    
    # Verifica se é necessario adicionar variaveis artificiais
    sol_basica = any(x == -1 for x in basicas(A)[1])
    if not sol_basica:
        d = (False, nvar)
        return c, A, b, d

    # Custos da nova função
    # Zero para as variáveis originais
    # E um para as variáveis artificiais
    c1 = [0] * len(c)
    id = basicas(A)[0]
    for i in range(0, len(id)):
        if id[i] == 0:
            coluna = [0] * len(id)
            coluna[i] = 1
            for linha in range(0, len(A)):
                A[linha].append(coluna[linha])
            c1.append(1)

    # Flag se precisou adicionar variaveis
    d = (True, nvar)

    return c1, A, b, d

##### Resultado da fase 1
Problema com variáveis artificiais: com os novos coeficientes, aplicamos o Simplex. A solução ótima da fase 1 nos dá uma solução básica viável do problema original. 

O tableau final da fase 1 é utilizado como base para retornar ao problema original. As colunas usadas são:

    - da funcao objetivo e valores das basicas
    - das variaveis originais
    
Antes, verificamos se:

- temos restrições LD
- a solução é degenerada
- o problema é infactível (função objetivo diferente de zero)

In [61]:
# Ajusta o tableau para retornar ao problema original após a Fase 1.
# Remove as variáveis artificiais do tableau.
# Retorna a matriz A e o vetor b ajustados para a Fase 2.
def resultado_fase1(tableau, nvar):
    # Mantem as colunas das variaveis originais
    if not is_zero(tableau[0][0]):
        return None, None

    qtdd_colunas = nvar + 1

    # Verifica se o tableau tem dimensões válidas
    if len(tableau) < 2:
        return None, None

    # Obtem os indices das variaveis basicas
    basic_vars = tableau[-1] if len(tableau[-1]) > 0 else []
    novo_A = []
    novo_b = []

    # Processa cada linha, exceto a primeira (custos) e a última (indices)
    for basica in range(1, len(tableau)-1):
        if basica < len(tableau) and basica < len(basic_vars) and basic_vars[basica] > nvar-1:
            for j in range(1, nvar+1):
                if j < len(tableau[basica]) and not is_zero(tableau[basica][j]):
                    tableau = atualiza_tableau(tableau, (basica, j))

    # Mantem as linhas das variaveis basicas originais
    if len(basic_vars) > 0:
        for i, xB in enumerate(basic_vars):
            if i+1 < len(tableau) and xB < qtdd_colunas:
                row = tableau[i+1]
                if len(row) >= qtdd_colunas:
                    novo_A.append(row[1:qtdd_colunas])
                    novo_b.append(row[0])

    return novo_A, novo_b

## 2- Fase 2: Aplicar Método Simplex

##### Tableau Inicial 

Para montar o tableau a matriz A deve conter todas as colunas da identidade necessárias para obter uma solução básica viável. Garantimos isso pois todos os problemas passam pela fase 1.

In [62]:
# Constrói o tableau inicial do método simplex.
# Inclui a linha de custos reduzidos, restrições e índices das variáveis básicas.
# Retorna o tableau inicial.
def tableau_inicial(c, A, b):
    # Encontra as variáveis básicas
    identidade, indice_basicas = basicas(A)
    
    # LZ: linha zero, contem o valor da função objetivo (fo) seguido pelos custos reduzidos
    # Função objetivo
    fo = -sum(b[i] * c[indice_basicas[i]] for i in range(len(b)))
    
    # Custos reduzidos
    cr = [ c[i] - sum(A[j][i] * c[indice_basicas[j]] 
            for j in range(len(A))) if i not in indice_basicas else 0
            for i in range(len(A[0]))
         ]
    lz = [fo] + cr
    
    # Próximas linhas do tableau: b | A
    linhas = [[b[i]] + A[i] for i in range(len(b))]
    
    # Inclui o indice das basicas na ultima linha do tableau
    return [lz] + linhas + [indice_basicas]

Funções auxiliares para atualizar o tableau:

In [63]:
# Verifica se o tableau atual ainda contém custos reduzidos negativos.
# Retorna True se a solução não é ótima, caso contrário False.
def nao_otima(tableau):
    return any(c < 0 for c in tableau[0][1:])

In [64]:
# Determina a posição do pivô para atualização do tableau.
# Identifica qual variável entra e qual sai da base.
def posicao_pivo(tableau):
    custos = tableau[0]
    
    # Escolhe o primeiro custo reduzido negativo (quem entra na base)
    for j in range(1,len(custos)):
        if custos[j] < 0:
            coluna_pivo = j
            break

    theta = []
    for linha_i in range (1,len(tableau)-1):
        if tableau[linha_i][coluna_pivo] <= 0:
            theta.append(math.inf)
        else:
            theta.append(tableau[linha_i][0] / tableau[linha_i][coluna_pivo])
    
    # Verifica se o problema é ilimitado
    if set(theta) == {math.inf}:
        print("Problema ilimitado")
        return None, None

    # Escolhe quem sai da base (menor indice)
    # Soma um porque estamos ignorando a linha de custos
    linha_pivo = theta.index(min(theta)) + 1
    
    return linha_pivo, coluna_pivo

In [65]:
# Atualiza o tableau com base no pivô escolhido.
# Realiza operações de linha para garantir que o pivô se torne 1 e zera os demais elementos na coluna.
def atualiza_tableau(tableau, pivo):
    i, j = pivo
    valor_pivo = tableau[i][j]
    
    # Divide a linha do pivo para que o valor do pivo seja 1
    tableau[i] = np.array(tableau[i]) / valor_pivo
    
    # Atualiza os valores de dentro do tableau
    # (zera os outros elementos da coluna do pivo)
    for linha in range (len(tableau)-1):
        if linha != i:
            multiplicador = (-1) * tableau[linha][j]
            tableau[linha] = multiplicador*tableau[i] + tableau[linha]

            tableau[linha] = np.array([0 if is_zero(x) else x for x in list(tableau[linha])])
   
    # Atualiza variaveis básicas
    tableau[-1][i-1] = j-1 

    return tableau 

 ##### Simplex  e  Solução ótima
 Funções auxiliares para o simplex de duas fases:

In [66]:
# Obtém a solução ótima do problema a partir do tableau final.
# Retorna o vetor solução contendo os valores das variáveis básicas.
def otima(tableau, nvar):
    solucoes = [0] * nvar  # Inicia com todos valendo zero
    
    # Coloca o valor das basicas na posicao correta da solucao
    for i, xB in enumerate(tableau[-1]):
        solucoes[xB] = tableau[i+1][0]
        
    return solucoes 

In [67]:
# Implementa o método simplex para resolver problemas de programação linear.
# Retorna a solução ótima ou informa se o problema é ilimitado ou infactível.
def simplex(c, A, b, d):
    tableau = tableau_inicial(c, A, b)
    nvar = d[1]
    it = 0
    
    while nao_otima(tableau) and it < 10000:
        it = it +1
        pivo = posicao_pivo(tableau)
        if pivo == (None, None):
            return None, None
        tableau = atualiza_tableau(tableau, pivo)

    # Iterações
    if d[0]:
        print("Iterações da fase 1: ",it)
    else:
        print("Iterações da fase 2: ",it)  
    if it == 10000:
        print("Max Iteracoes atingida")
        return None, None

    # Se estamos na Fase 1 retorna um tableau
    if d[0]:
        novo_A, novo_b = resultado_fase1(tableau, nvar)
        return novo_A, novo_b
    # Se estamos na Fase 2 retorna a solução ótima
    else:
        return A, otima(tableau, nvar) 

## 3- Simplex de duas fases:

In [68]:
# Resolve um problema de programação linear utilizando o método simplex,
# começando pela Fase 1 se necessário.
def get_solution_simplex(c, A, b):
    inicio = time.time()
    nvar = len(A[0])
    c1, A1, b1, d = fase1(c, A, b)

    # Caso seja necessario usar variaveis artificiais (Fase 1)
    if d[0]:
        A2, b2 = simplex(c1,A1,b1,d)
        if A2 is None:
            print("Problema infactivel")
            return None

        # Convertendo tudo para mesmo formato que o simplex (Fase 2)
        A2 = [[float(x) for x in row] for row in A2]
        b2 = [float(x) for x in b2] 
        
        fA, fb = simplex(c, A2, b2,(False, nvar))
        print("Solução ótima: ",fb)
        
    # Não foi necessario usar a fase 1 (Resolve direto Fase 2)    
    else:  
        fA, fb = simplex(c, A1, b1, d)
        print("Solução ótima: ", fb)
    
    fim = time.time()
    print("Tempo de execução: ", fim - inicio)

## 4- Testando alguns exemplos

#### Slides da aula 23

Solucão básica viável para iniciar o simplex:
(x1,x2,x3,x4) = (1 ; 1/2 ; 1/3 ; 0) e zero nas variaveis artificiais

Problema: A tem linhas LD --> sol. encontrada na fase 1 é degenerada

In [69]:
# Coeficientes da função objetivo
c = [1, 1, 1, 0]

# Matriz de coeficientes das restrições
A = [
    [1, 2, 3, 0],
    [-1, 2, 6, 0],
    [0, 4, 9, 0],
    [0, 0, 3, 1]
]

# Vetor de termos independentes das restrições
b = [3, 2, 5, 1]

get_solution_simplex(c, A, b)

Iterações da fase 1:  2
Iterações da fase 2:  1
Solução ótima:  [np.float64(0.5), np.float64(1.2500000000000002), 0, np.float64(1.0000000000000002)]
Tempo de execução:  0.0005404949188232422


#### Lista 18
Solução ótima esperada: (3,2,0,0)

In [70]:
# Coeficientes da função objetivo
c = [-2, -1, 0, 0] 

# Matriz de coeficientes das restrições
A = [
    [-1, 1, -1, 0],  
    [1, 2, 0, 1]   
]

# Vetor de termos independentes das restrições
b = [-1, 7]

get_solution_simplex(c, A, b)

Iterações da fase 1:  1
Iterações da fase 2:  1
Solução ótima:  [np.float64(3.0), np.float64(2.0), 0, 0]
Tempo de execução:  0.0009050369262695312


#### Lista 20
Solução ótima esperada: (1,5,0)

In [71]:
# Coeficientes da função objetivo
c = [4, 3, 7] 

# Matriz de coeficientes das restrições
A = [
    [2, 2, 1], 
    [3, 1, 2]  
]

# Vetor de termos independentes das restrições
b = [12, 8]

get_solution_simplex(c, A, b)

Iterações da fase 1:  2
Iterações da fase 2:  0
Solução ótima:  [1.0, 5.0, 0]
Tempo de execução:  0.00029015541076660156


#### Lista 20
Trocando b para (5,8)

Solução ótima esperada: (2,0,1)

In [72]:
# Coeficientes da função objetivo
c = [4, 3, 7]

# Matriz de coeficientes das restrições
A = [
    [2, 2, 1], 
    [3, 1, 2]  
]

# Vetor de termos independentes das restrições
b = [5, 8]

get_solution_simplex(c, A, b)

Iterações da fase 1:  2
Iterações da fase 2:  0
Solução ótima:  [2.0, 0, 1.0]
Tempo de execução:  0.00028014183044433594


#### Lista 16
Solução esperada: problema infactível

In [73]:
# Coeficientes da função objetivo
c = [2, -1, 5, 0] 

# Matriz de coeficientes das restrições
A = [
    [1, 1, 1, 0], 
    [2, 3, 2, 1] 
]

# Vetor de termos independentes das restrições
b = [4, 5]

get_solution_simplex(c, A, b)

Iterações da fase 1:  1
Problema infactivel


#### Lista 13
Solução esperada: x1 = -1/3  e  x2 = 10/3

OBS: x1 livre --> (x1+) - (x1-)

In [74]:
# Coeficientes da função objetivo
#x1+, x1-, x2, x3, x4, x5
c = [-1, 1, 1, 0, 0, 0] 

# Matriz de coeficientes das restrições
A = [
    [1, -1, 1, -1, 0, 0], 
    [-1, 1, 1, 0, 1, 0],
    [-2, 2, 1, 0, 0, -1]
]

# Vetor de termos independentes das restrições
b = [3, 4, 4]

get_solution_simplex(c, A, b)

Iterações da fase 1:  2
Iterações da fase 2:  0
Solução ótima:  [0, 0.33333333333333326, 3.3333333333333335, 0, 0.33333333333333326, 0]
Tempo de execução:  0.0004706382751464844


#### Lista 19
Solução ótima esperada: (2, 0, 1, 0, 0)

In [75]:
# Coeficientes da função objetivo
c = [-3, 3, -4, 4, 0] 

# Matriz de coeficientes das restrições
A = [
    [1, -1, 0, 0, 1], 
    [0, 0, 1, -1, 1]
]

# Vetor de termos independentes das restrições
b = [2, 1]

get_solution_simplex(c, A, b)

Iterações da fase 2:  0
Solução ótima:  [2, 0, 1, 0, 0]
Tempo de execução:  0.00041866302490234375


#### Lista 15 - Verificar troca de base usando Regra de Bland
Solução ótima esperada: (0, 8, 0, 0)

In [76]:
# Coeficientes da função objetivo
c = [-2, -3, 0, 0]

# Matriz de coeficientes das restrições
A = [
    [2, 1, 1, 0],
    [3, 3, 0, 1]
]

# Vetor de termos independentes das restrições
b = [8, 24]

get_solution_simplex(c,A,b)

Iterações da fase 2:  2
Solução ótima:  [0, np.float64(8.0), 0, np.float64(0.0)]
Tempo de execução:  0.0002200603485107422


---

In [80]:
import torch

def print_matrix(matrix):
    for row in matrix:
        print(row)


nvar = 10

A = torch.round(torch.rand(nvar, nvar), decimals=2)  
C = torch.round(torch.rand(nvar, 1), decimals=2)  
X = torch.round(torch.rand(nvar, 1), decimals=2)  


A1 = A[:int(0.2*nvar), :]
A2 = A[:int(0.5*nvar), :]
A3 = A[:int(0.8*nvar), :]


b1 = A1@X
b2 = A2@X
b3 = A3@X



print("Dimensões:")
print(f"A: {len(A)}x{len(A[0])}")
print(f"X: {len(X)}x{len(X[0])}")
print(f"A1: {len(A1)}x{len(A1[0])}")
print(f"A2: {len(A2)}x{len(A2[0])}")
print(f"A3: {len(A3)}x{len(A3[0])}")
print(f"b1: {len(b1)}x{len(b1[0])}")
print(f"b2: {len(b2)}x{len(b2[0])}")
print(f"b3: {len(b3)}x{len(b3[0])}")
print(f"C: {len(C)}x{len(C[0])}")

# Convert tensors to float lists
A = A.tolist()
A1 = A1.tolist()
A2 = A2.tolist()
A3 = A3.tolist()
C = C.T.tolist()
b1 = b1.T.tolist()
b2 = b2.T.tolist()
b3 = b3.T.tolist()

# Convert nested lists to float
print_matrix(A1)
print_matrix(A2)
print_matrix(A3)


get_solution_simplex(C, A1, b1)

#get_solution_simplex(C, A2, b2)
#get_solution_simplex(C, A3, b3)

#--------------------------------

#pegar solucao do Linprog







Dimensões:
A: 10x10
X: 10x1
A1: 2x10
A2: 5x10
A3: 8x10
b1: 2x1
b2: 5x1
b3: 8x1
C: 10x1
[0.8100000023841858, 0.4699999988079071, 0.6299999952316284, 0.6000000238418579, 0.11999999731779099, 0.5099999904632568, 0.8199999928474426, 0.8799999952316284, 0.27000001072883606, 0.3199999928474426]
[0.7699999809265137, 0.949999988079071, 0.6100000143051147, 0.029999999329447746, 0.3499999940395355, 0.03999999910593033, 0.8999999761581421, 0.7799999713897705, 0.38999998569488525, 0.4399999976158142]
[0.8100000023841858, 0.4699999988079071, 0.6299999952316284, 0.6000000238418579, 0.11999999731779099, 0.5099999904632568, 0.8199999928474426, 0.8799999952316284, 0.27000001072883606, 0.3199999928474426]
[0.7699999809265137, 0.949999988079071, 0.6100000143051147, 0.029999999329447746, 0.3499999940395355, 0.03999999910593033, 0.8999999761581421, 0.7799999713897705, 0.38999998569488525, 0.4399999976158142]
[0.12999999523162842, 0.2199999988079071, 0.6600000262260437, 0.30000001192092896, 0.82999998331069

IndexError: list index out of range